<a href="https://colab.research.google.com/github/ryanvuu/csc-365-lab7/blob/main/csc_365_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
username = os.getenv("username")

Install libraries

In [ ]:
pip install mysql-connector-python

In [ ]:
import getpass
db_password = getpass.getpass()
# print(db_password)

In [ ]:
import mysql.connector
conn = mysql.connector.connect(user=username, password=db_password,
                               host='mysql.labthreesixfive.com',
                               database='rvu03')

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM lab7_rooms")
result = cursor.fetchall()
print(result)

### FR1: Rooms and Rates ###

In [ ]:
pass

# **Main Program**

In [ ]:
def main():
  while True:
    print("Select a Reservation Option:")
    print("[1] Rooms and Rates")
    print("[2] Reservations")
    print("[3] Reservation Cancellation")
    print("[4] Detailed Reservation Information")
    print("[5] Revenue")
    print("[6] Quit")

    choice = input()
    if choice == "1":
      cursor = conn.cursor()
      cursor.execute("""
                      WITH PopTable AS (
                        SELECT lab7_rooms.RoomCode,
                            lab7_rooms.RoomName,
                            lab7_rooms.Beds,
                            lab7_rooms.bedType,
                            lab7_rooms.maxOcc,
                            lab7_rooms.basePrice,
                            lab7_rooms.decor,
                            ROUND(SUM(COALESCE(DATEDIFF(lab7_reservations.Checkout, lab7_reservations.CheckIn), 0)) / 180, 2) AS PopScore
                        FROM lab7_rooms
                        LEFT JOIN lab7_reservations
                            ON lab7_rooms.RoomCode = lab7_reservations.Room
                            AND lab7_reservations.CheckIn >= DATE_SUB(CURDATE(), INTERVAL 180 DAY)
                        GROUP BY lab7_rooms.RoomCode
                      ), NextAvailable AS (
                        SELECT RoomCode,
                        CASE WHEN
                          -- If a reservation goes past the current date
                          EXISTS (
                            SELECT *
                            FROM lab7_reservations
                            WHERE lab7_reservations.Room = lab7_rooms.RoomCode
                                AND lab7_reservations.CheckIn <= CURDATE()
                                AND lab7_reservations.Checkout >= CURDATE()
                          ) THEN (
                            -- Get the earliest checkout date that occurred after the current date
                            SELECT MIN(first_res.Checkout)
                            FROM lab7_reservations AS first_res
                            WHERE first_res.Room = lab7_rooms.RoomCode
                              AND first_res.Checkout > CURDATE()
                              -- See if another guest checked in on the date that the prev. reservaton checked out
                              AND NOT EXISTS (
                                SELECT *
                                FROM lab7_reservations as second_res
                                WHERE first_res.Room = second_res.Room
                                  AND second_res.CheckIn = first_res.Checkout
                              )
                          )
                          ELSE CURDATE()
                        END AS NextAvailableCheckIn
                        FROM lab7_rooms
                      ), RecentReservations AS (
                        SELECT RoomCode,
                        DATEDIFF(
                          (
                            SELECT MAX(Checkout)
                            FROM lab7_reservations
                            WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.Checkout <= CURDATE()
                          ),
                          (
                            SELECT MAX(Checkin)
                            FROM lab7_reservations
                            WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                              AND lab7_reservations.Checkout <= CURDATE() 
                          )
                        ) AS MostRecDuration,
                        (
                          SELECT MAX(Checkout)
                          FROM lab7_reservations
                          WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                            AND lab7_reservations.Checkout <= CURDATE()
                        ) AS MostRecentCheckout
                        FROM lab7_rooms
                      )
                      SELECT *
                      FROM PopTable
                      NATURAL JOIN NextAvailable
                      NATURAL JOIN RecentReservations
                      ORDER BY PopScore DESC;
                     """)
      result = cursor.fetchall()
      print(result)
    elif choice == "2":
      pass
    elif choice == "3":
      pass
    elif choice == "4":
      pass
    elif choice == "5":
      pass
    elif choice == "6":
      break


In [ ]:
if __name__ == "__main__":
    main()